In [1]:
from __future__ import annotations

from pathlib import Path
import json

import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
from branca.element import MacroElement, Template, Element


# -----------------------------
# Pfade / Output
# -----------------------------



def find_project_root(start: Path) -> Path:
    """
    Geht von 'start' nach oben, bis ein Ordner gefunden wird,
    der wie dein Projektroot aussieht (data/ und outputs/ vorhanden).
    """
    for p in [start, *start.parents]:
        if (p / "data").is_dir() and (p / "outputs").is_dir():
            return p
    raise RuntimeError(
        f"Projekt-Root nicht gefunden ab: {start}. "
        "Starte das Notebook innerhalb des Projekts oder passe die Erkennung an."
    )

PROJECT_ROOT = find_project_root(Path.cwd())

LETTERS_CSV = PROJECT_ROOT / "data" / "processed" / "letters_master.csv"

OUT_DIR = PROJECT_ROOT / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_HTML = OUT_DIR / "romkorr_map.html"

CENTER = (51.0, 10.0)
ZOOM_START = 6

TOP_N_PERSONS = 80
EXAMPLES_PER_PLACE = 6
PAGE_SIZE_RESULTS = 80
COORD_ROUND = 5


# -----------------------------
# Helper
# -----------------------------
def to_de_date(date_iso: str) -> str:
    """ISO YYYY-MM-DD -> DD.MM.YYYY. Sonst Original zurück."""
    if not date_iso or not isinstance(date_iso, str):
        return ""
    parts = date_iso.split("-")
    if len(parts) >= 3 and all(p.isdigit() for p in parts[:3]):
        y, m, d = parts[:3]
        if len(y) == 4:
            return f"{d.zfill(2)}.{m.zfill(2)}.{y}"
    return date_iso


def coord_key(lat: float, lon: float, ndigits: int = COORD_ROUND) -> str:
    return f"{round(float(lat), ndigits):.{ndigits}f},{round(float(lon), ndigits):.{ndigits}f}"


# -----------------------------
# Daten laden + Normalisieren
# -----------------------------
letters = pd.read_csv(LETTERS_CSV)

# Normalize column casing
rename_map = {}
for c in letters.columns:
    if c.lower() == "sender":
        rename_map[c] = "Sender"
    if c.lower() == "recipient":
        rename_map[c] = "Recipient"
letters = letters.rename(columns=rename_map)

if "date_iso" not in letters.columns:
    letters["date_iso"] = ""
letters["date_iso"] = letters["date_iso"].fillna("").astype(str)
letters["date_de"] = letters["date_iso"].map(to_de_date)

need_cols = {
    "dispatch_lat", "dispatch_lon",
    "destination_lat", "destination_lon",
    "Sender", "Recipient",
    "link_canonical", "date_iso",
}
missing = [c for c in sorted(need_cols) if c not in letters.columns]
if missing:
    raise RuntimeError(f"Missing columns in letters_master.csv: {missing}")


# -----------------------------
# Top Personen (Dropdown)
# -----------------------------
persons = pd.concat(
    [letters["Sender"].dropna().astype(str), letters["Recipient"].dropna().astype(str)],
    ignore_index=True,
)
persons = persons[persons.str.len() > 0]
top_persons = persons.value_counts().head(TOP_N_PERSONS).index.tolist()


# -----------------------------
# Ort -> Briefe Index (für Hotspots + Ort-Dropdown)
# -----------------------------
place_index: dict[str, list[dict]] = {}


def add_place_entries(lat_col: str, lon_col: str, place_col: str | None) -> None:
    sub = letters.dropna(subset=[lat_col, lon_col]).copy()
    sub["k"] = [coord_key(a, b) for a, b in zip(sub[lat_col], sub[lon_col])]
    sub = sub.sort_values(["date_iso"], na_position="last")

    for _, r in sub.iterrows():
        k = r["k"]
        place_index.setdefault(k, []).append(
            {
                "s": str(r.get("Sender", "?")),
                "r": str(r.get("Recipient", "?")),
                "d": str(r.get("date_de", "")),
                "iso": str(r.get("date_iso", "")) if pd.notnull(r.get("date_iso")) else "",
                "u": str(r.get("link_canonical", "#")),
                "id": int(r.get("letter_id"))
                if ("letter_id" in r and pd.notnull(r.get("letter_id")))
                else None,
                "place": str(r.get(place_col, "")) if place_col and place_col in r else "",
            }
        )


add_place_entries("dispatch_lat", "dispatch_lon", "place_dispatch")
add_place_entries("destination_lat", "destination_lon", "place_destination")


# -----------------------------
# Hotspots / Heatmap Aggregation
# -----------------------------
def build_place_agg() -> pd.DataFrame:
    rows = []
    for lat_col, lon_col, name_col in [
        ("dispatch_lat", "dispatch_lon", "place_dispatch"),
        ("destination_lat", "destination_lon", "place_destination"),
    ]:
        tmp = letters.dropna(subset=[lat_col, lon_col]).copy()
        tmp["k"] = [coord_key(a, b) for a, b in zip(tmp[lat_col], tmp[lon_col])]
        tmp["lat"] = tmp[lat_col].astype(float)
        tmp["lon"] = tmp[lon_col].astype(float)
        tmp["place"] = tmp[name_col].astype(str) if name_col in tmp.columns else "Ort (unbekannt)"
        rows.append(tmp[["k", "lat", "lon", "place"]])

    if not rows:
        return pd.DataFrame(columns=["k", "lat", "lon", "w", "place"])

    allp = pd.concat(rows, ignore_index=True)

    def mode_place(s: pd.Series) -> str:
        s = s.dropna()
        if s.empty:
            return "Ort (unbekannt)"
        m = s.mode()
        return str(m.iloc[0]) if not m.empty else str(s.iloc[0])

    agg = allp.groupby("k", as_index=False).agg(
        lat=("lat", "mean"),
        lon=("lon", "mean"),
        w=("k", "size"),
        place=("place", mode_place),
    )
    return agg


agg = build_place_agg()
place_name_by_key = dict(zip(agg["k"], agg["place"]))

# payload für JS: key -> { place, items[] }
place_payload = {
    k: {"place": place_name_by_key.get(k, "Ort (unbekannt)"), "items": v}
    for k, v in place_index.items()
}

# -----------------------------
# Linien Payload (alle Briefe)
# + zusätzlich: ak/bk = koordinaten-key für Ort-Filter
# -----------------------------
lines_df = letters.dropna(
    subset=["dispatch_lat", "dispatch_lon", "destination_lat", "destination_lon"]
).copy()

lines_payload = []
for r in lines_df.itertuples(index=False):
    iso = getattr(r, "date_iso", "")
    iso = "" if iso is None else str(iso)

    a_lat = float(getattr(r, "dispatch_lat"))
    a_lon = float(getattr(r, "dispatch_lon"))
    b_lat = float(getattr(r, "destination_lat"))
    b_lon = float(getattr(r, "destination_lon"))

    lines_payload.append(
        {
            "a": [a_lat, a_lon],
            "b": [b_lat, b_lon],
            "ak": coord_key(a_lat, a_lon),   # origin place-key
            "bk": coord_key(b_lat, b_lon),   # destination place-key
            "s": str(getattr(r, "Sender")),
            "r": str(getattr(r, "Recipient")),
            "d": to_de_date(iso),
            "iso": iso,
            "km": float(getattr(r, "distance_km"))
            if hasattr(r, "distance_km") and pd.notnull(getattr(r, "distance_km"))
            else None,
            "u": str(getattr(r, "link_canonical")),
            "id": int(getattr(r, "letter_id"))
            if hasattr(r, "letter_id") and pd.notnull(getattr(r, "letter_id"))
            else None,
        }
    )

# -----------------------------
# Ort-Counts passend zur Filter-Logik
# (zählt so wie JS-Filter: Anzahl Linien, nicht "unique ids")
# -----------------------------
place_counts_lines: dict[str, int] = {}

for x in lines_payload:
    keys = {x.get("ak"), x.get("bk")}  # set -> verhindert Doppelzählung falls ak==bk
    keys.discard(None)
    keys.discard("")
    for k in keys:
        place_counts_lines[k] = place_counts_lines.get(k, 0) + 1

# Ort-Optionen für Dropdown (sortiert nach Name)
place_options = [
    {"k": k, "p": place_name_by_key.get(k, "Ort (unbekannt)"), "n": int(place_counts_lines.get(k, 0))}
    for k in place_index.keys()
]
place_options = sorted(place_options, key=lambda x: (x["p"].lower(), x["k"]))


# -----------------------------
# Folium Karte bauen
# -----------------------------
m = folium.Map(location=CENTER, zoom_start=ZOOM_START, control_scale=True)

# Heatmap
heat_fg = folium.FeatureGroup(name="Heatmap", show=False).add_to(m)
w_scaled = np.sqrt(agg["w"].astype(float)) if len(agg) else np.array([])
heat_points = [
    [float(r.lat), float(r.lon), float(ws)]
    for r, ws in zip(agg.itertuples(index=False), w_scaled)
]
HeatMap(
    heat_points,
    radius=37,
    blur=35,
    min_opacity=0.35,
    max_zoom=7,
).add_to(heat_fg)

# Hotspots
cluster = MarkerCluster(name="Hotspots", show=True).add_to(m)


def popup_html_for_place(k: str, place: str, n_total: int, ex_items: list[dict]) -> str:
    ex_li = []
    for it in ex_items:
        who = f"{it.get('s','?')} → {it.get('r','?')}"
        d = it.get("d", "")
        u = it.get("u", "#")
        ex_li.append(f"<li><a href='{u}' target='_blank' rel='noopener'>{who} ({d})</a></li>")

    btn = (
        "<button "
        f"onclick=\"window.rkOpenPlace && window.rkOpenPlace('{k}')\" "
        "style='padding:6px 10px;border:1px solid #ddd;border-radius:8px;background:#fff;cursor:pointer;'>"
        "Alle Korrespondenzen</button>"
    )

    return (
        "<div style='font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;'>"
        f"<div style='font-weight:650;margin-bottom:4px;'>{place}</div>"
        f"<div style='opacity:.8;margin-bottom:8px;'>Briefe am Ort: {n_total}</div>"
        f"<div style='margin: 8px 0;'>{btn}</div>"
        "<div><b>Beispiele:</b></div>"
        "<ul style='padding-left: 18px; margin: 6px 0;'>"
        + ("".join(ex_li) if ex_li else "<li>Keine Beispiele verfügbar.</li>")
        + "</ul></div>"
    )


for r in agg.itertuples(index=False):
    k = coord_key(r.lat, r.lon)
    items = place_index.get(k, [])
    count_for_k = place_counts_lines.get(k, 0)
    html = popup_html_for_place(k, str(r.place), count_for_k, items[:EXAMPLES_PER_PLACE])
    folium.Marker(
        location=(float(r.lat), float(r.lon)),
        popup=folium.Popup(html, max_width=420),
        tooltip=f"{r.place} ({count_for_k})" if count_for_k else str(r.place),
    ).add_to(cluster)

# Linienlayer
lines_group = folium.FeatureGroup(name="Korrespondenzen", show=True).add_to(m)


# -----------------------------
# UI HTML (Sidebar + Modal)
# -----------------------------
ui_block = """<style>
  canvas.leaflet-heatmap-layer { pointer-events: none !important; }
  .rk-sidebar{
    position:absolute;
    top:10px;
    left:10px;
    z-index:9999;
    background:rgba(255,255,255,0.94);
    padding:10px 12px;
    border-radius:12px;
    box-shadow:0 2px 12px rgba(0,0,0,.18);
    width:360px;
    font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;

    /* NEU */
    max-height: calc(100vh - 20px);
    overflow: auto;
  }
  .rk-title{ margin:0 0 8px 0; font-size:14px; font-weight:650; text-align:center; }
  .rk-row{ display:flex; gap:8px; align-items:center; margin:6px 0; }
  .rk-row select, .rk-row input{ width:100%; padding:6px; border-radius:10px; border:1px solid #ddd; }
  .rk-row button{ padding:6px 10px; border-radius:10px; border:1px solid #ddd; background:#fff; cursor:pointer; }
  .rk-row label{ font-size:12px; display:flex; gap:8px; align-items:center; }
  .rk-meta{ font-size:12px; opacity:.85; margin-top:6px; line-height:1.35; }
  .rk-divider{ height:1px; background:#eee; margin:10px 0; }
  .rk-list{ max-height:320px; overflow:auto; border:1px solid #eee; border-radius:10px; padding:6px; background:#fff; }
  .rk-list ul{ margin:0; padding-left:18px; }
  .rk-list li{ margin:4px 0; font-size:12px; line-height:1.25; }
  .rk-pager{ display:flex; justify-content:space-between; align-items:center; margin-top:6px; font-size:12px; }
  .rk-muted{ opacity:.75; }
  .rk-toggle{
    position:absolute; top:10px; left:10px; z-index:10001;
    padding:8px 10px; border-radius:12px; border:1px solid #ddd;
    background:rgba(255,255,255,0.92); box-shadow:0 2px 12px rgba(0,0,0,.18);
    font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;
    cursor:pointer;
  }
  .rk-modal-backdrop{ position:absolute; inset:0; z-index:10000; background:rgba(0,0,0,.45); display:none;
    align-items:center; justify-content:center; padding:16px; }
  .rk-modal{ width:min(720px,96vw); background:rgba(255,255,255,0.98); border-radius:16px;
    box-shadow:0 8px 28px rgba(0,0,0,.25); padding:14px 16px; font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif; }
  .rk-modal h3{ margin:4px 0 8px 0; font-size:16px; }
  .rk-modal p{ margin:6px 0; font-size:13px; line-height:1.45; }
</style>

<button id="rkSidebarToggle" class="rk-toggle" type="button">×</button>

<div class="rk-sidebar">
  <div class="rk-title">Filter & Liste</div>

  <div class="rk-row">
    <label><input type="checkbox" id="rkShowLines" checked /> Korrespondenzen</label>
  </div>

  <div class="rk-row">
    <select id="rkPerson"><option value="">(alle Personen)</option></select>
    <button id="rkReset" title="Filter zurücksetzen">Reset</button>
  </div>

  <div class="rk-row">
    <select id="rkPlace"><option value="">(alle Orte)</option></select>
  </div>

  <div class="rk-row">
    <input id="rkYearFrom" type="number" inputmode="numeric" placeholder="Jahr von" />
    <input id="rkYearTo" type="number" inputmode="numeric" placeholder="Jahr bis" />
  </div>

  <div class="rk-row" style="justify-content: space-between;">
    <button id="rkInfo" title="Info">Info</button>
    <div class="rk-muted" style="font-size:12px;">Tipp: Hotspot → „Alle Korrespondenzen“</div>
  </div>

  <div class="rk-meta">
    Korrespondenzen: <span id="rkLineCount">0</span> (von <span id="rkLineTotal">0</span>)
  </div>

  <div class="rk-divider"></div>

  <!-- Ergebnisliste: IMMER sichtbar -->
  <div id="rkResultsBox">
    <div class="rk-row" style="justify-content:space-between; align-items:center;">
      <h4 id="rkResultsTitle" style="margin:0; font-size:13px;">
        Gefilterte Briefe
      </h4>
      <button id="rkDownloadResults">CSV</button>
    </div>

    <div class="rk-meta" id="rkResultsMeta"></div>
    <div class="rk-list"><ul id="rkResultsList"></ul></div>
    <div class="rk-pager">
      <button id="rkResPrev">‹</button>
      <span class="rk-muted" id="rkResPageInfo">–</span>
      <button id="rkResNext">›</button>
    </div>
  </div>
</div>

<div class="rk-modal-backdrop" id="rkModal">
  <div class="rk-modal" role="dialog" aria-modal="true" aria-labelledby="rkModalTitle">
    <h3 id="rkModalTitle">Korrespondenzen der Frühromantik</h3>
    <p>Diese Karte gibt Einblick in die Korrespondenzen (Briefe) der Frühromantik. Die Daten stammen von https://briefe-der-romantik.de/. Es wurden nur Korrespondenzen abgebildet, die einen Absende- oder Empfangsort haben.</p>
    <p><b>Filter:</b> Die Briefe lassen sich anhand Person, Ort und Jahr filtern. Im Fenster "Filter & Liste" sieht man die gefilterten Briefe. Sie können die Liste als .csv herunterladen. Die Briefe lassen sich auch in der Karte über „Alle Korrespondenzen“ filtern, wenn ein Marker angewählt ist.</p>
    <p><b>Heatmap und Hotspots :</b> Diese geben Aufschluss darüber, wo viele Briefe versendet und empfangen wurden.</p>
    <div class="rk-row" style="justify-content:flex-end;">
      <button id="rkModalClose">Schließen</button>
    </div>
  </div>
</div>
"""
m.get_root().html.add_child(Element(ui_block))


# -----------------------------
# JS Macro
# -----------------------------
template = Template(r"""
{% macro html(this, kwargs) %}
<script>
(function() {
  const MAP_NAME = "{{ this.map_name }}";
  const LINES = {{ this.lines_json }};
  const PERSONS = {{ this.persons_json }};
  const PLACES = {{ this.places_json }};
  const PLACE_OPTIONS = {{ this.place_options_json }};
  const PAGE_SIZE = {{ this.page_size }};
  const LINES_GROUP = "{{ this.lines_group_name }}";

  function safeText(s) { return (s || "").toString(); }

  function yearFromIso(iso) {
    const m = (iso || "").toString().match(/^(\d{4})/);
    return m ? parseInt(m[1], 10) : null;
  }

  function passesYearFilter(iso, yFrom, yTo){
    const y = yearFromIso(iso);
    if (y == null) return true;
    if (yFrom && y < yFrom) return false;
    if (yTo && y > yTo) return false;
    return true;
  }

  function passesPlaceFilter(line, placeKey){
    if (!placeKey) return true;
    return (line.ak === placeKey || line.bk === placeKey);
  }

  function getMap(){ return window[MAP_NAME]; }

  function popupHtml(x) {
    const who = `${safeText(x.s)} → ${safeText(x.r)}`;
    const d = safeText(x.d);
    const km = (x.km == null) ? "" : ` · ${Math.round(x.km)} km`;
    const u = safeText(x.u || "#");
    return `<div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;">
      <div style="font-weight:650;margin-bottom:4px;">${who}</div>
      <div style="opacity:.8;margin-bottom:6px;">${d}${km}</div>
      <a href="${u}" target="_blank" rel="noopener">Brief öffnen</a>
    </div>`;
  }

  function downloadCsv(rows, filename) {
    const header = ["id","date","sender","recipient","url"];
    const lines = [header.join(",")].concat(rows.map(r => [
      r.id ?? "",
      (r.d ?? "").toString().replaceAll(",", " "),
      (r.s ?? "").toString().replaceAll(",", " "),
      (r.r ?? "").toString().replaceAll(",", " "),
      (r.u ?? "").toString().replaceAll(",", " "),
    ].join(",")));
    const blob = new Blob([lines.join("\n")], {type:"text/csv;charset=utf-8"});
    const a = document.createElement("a");
    a.href = URL.createObjectURL(blob);
    a.download = filename;
    a.click();
    URL.revokeObjectURL(a.href);
  }

  function getFilters(){
    const show = document.getElementById('rkShowLines').checked;
    const person = (document.getElementById('rkPerson').value || "").trim();
    const placeKey = (document.getElementById('rkPlace').value || "").trim();
    const yFrom = parseInt((document.getElementById("rkYearFrom").value || "").trim(), 10) || null;
    const yTo = parseInt((document.getElementById("rkYearTo").value || "").trim(), 10) || null;
    return { show, person, placeKey, yFrom, yTo };
  }

  function getFilteredLines(){
    const { person, placeKey, yFrom, yTo } = getFilters();
    const out = [];
    for (let i=0; i<LINES.length; i++){
      const x = LINES[i];
      if (person && x.s !== person && x.r !== person) continue;
      if (!passesYearFilter(x.iso, yFrom, yTo)) continue;
      if (!passesPlaceFilter(x, placeKey)) continue;
      out.push(x);
    }
    return out;
  }

  function drawLines() {
    const map = getMap();
    if (!map || !map.addLayer || !window.L) return;

    const { show } = getFilters();
    document.getElementById('rkLineTotal').textContent = LINES.length.toString();

    const host = window[LINES_GROUP];
    if (!host || !host.clearLayers) {
      console.error("Korrespondenzen-Layer nicht gefunden:", LINES_GROUP, host);
      return;
    }

    if (show) {
      if (!map.hasLayer(host)) map.addLayer(host);
    } else {
      host.clearLayers();
      if (map.hasLayer(host)) map.removeLayer(host);
      document.getElementById('rkLineCount').textContent = "0";
      return;
    }

    host.clearLayers();
    const filtered = getFilteredLines();

    let shown = 0;
    for (let i=0; i<filtered.length; i++){
      const x = filtered[i];
      // 1) Unsichtbare, dicke "Hitbox"-Linie (nur fürs Klicken)
      const hit = L.polyline([x.a, x.b], {
        weight: 10,          // <- Klick-Toleranz (z.B. 8–14 testen)
        opacity: 0,          // unsichtbar
        interactive: true,   // wichtig: soll Events bekommen
        smoothFactor: 1.5,
      });

      // 2) Sichtbare Linie (optisch wie bisher)
      const vis = L.polyline([x.a, x.b], {
        weight: 1,
        opacity: 0.35,
        smoothFactor: 1.5,
      });

      // Hover-Effekt → Sichtbare Linie hervorheben
      hit.on('mouseover', () => {
        vis.setStyle({ opacity: 0.65 });
      });

      hit.on('mouseout', () => {
        vis.setStyle({ opacity: 0.35 });
      });
                    
      // Popup an die Hitbox hängen (oder beide)
      hit.bindPopup(popupHtml(x), { maxWidth: 380 });

      // Beide in die Layer-Gruppe
      hit.addTo(host);
      vis.addTo(host);

      shown++;
    }
    document.getElementById('rkLineCount').textContent = shown.toString();
  }

  function renderResults(page){
    const title = document.getElementById('rkResultsTitle');
    const meta = document.getElementById('rkResultsMeta');
    const ul = document.getElementById('rkResultsList');
    const pageInfo = document.getElementById('rkResPageInfo');

    const { placeKey } = getFilters();
    const filtered = getFilteredLines();

    const placeName = placeKey && PLACES[placeKey] ? (PLACES[placeKey].place || "Ort") : "";
    title.textContent = placeName ? `Gefilterte Briefe – ${placeName}` : "Gefilterte Briefe";

    const n = filtered.length;
    const maxPage = Math.max(0, Math.ceil(n / PAGE_SIZE) - 1);
    const p = Math.min(Math.max(0, page), maxPage);
    const start = p * PAGE_SIZE;
    const end = Math.min(n, start + PAGE_SIZE);
    const slice = filtered.slice(start, end);

    meta.innerHTML = `<span style="opacity:.75">${n} Treffer</span> · Seite ${p+1} / ${maxPage+1}`;

    ul.innerHTML = "";
    slice.forEach(x=>{
      const who = `${safeText(x.s)} → ${safeText(x.r)}`;
      const d = safeText(x.d);
      const u = safeText(x.u||"#");
      const li = document.createElement("li");
      li.innerHTML = `<a href="${u}" target="_blank" rel="noopener">${who} (${d})</a>`;
      ul.appendChild(li);
    });

    pageInfo.textContent = (n===0) ? "keine Treffer" : `${start+1}–${end} von ${n}`;

    document.getElementById('rkResPrev').disabled = (p<=0);
    document.getElementById('rkResNext').disabled = (p>=maxPage);
    document.getElementById('rkResPrev').onclick = ()=>renderResults(p-1);
    document.getElementById('rkResNext').onclick = ()=>renderResults(p+1);

    document.getElementById('rkDownloadResults').onclick = ()=>downloadCsv(filtered, "romkorr_filter.csv");
  }

  // Hotspot-Button: setzt Ort-Dropdown und filtert
  window.rkOpenPlace = function(key){
    try{
      const sel = document.getElementById('rkPlace');
      if (sel) sel.value = key;
      applyFilters();
    }catch(e){ console.error(e); }
  };

  function applyFilters(){
    drawLines();
    renderResults(0);
  }

  function initUi(){
    const map = getMap();
    try { if (map && map.zoomControl) map.zoomControl.setPosition('bottomright'); } catch(e) {}

    // Person-Dropdown
    const personSel = document.getElementById('rkPerson');
    PERSONS.forEach(p=>{
      const o = document.createElement('option');
      o.value = p;
      o.textContent = p;
      personSel.appendChild(o);
    });

    // Ort-Dropdown
    const placeSel = document.getElementById('rkPlace');
    PLACE_OPTIONS.forEach(x=>{
      const o = document.createElement('option');
      o.value = x.k;
      o.textContent = `${x.p} (${x.n})`;
      placeSel.appendChild(o);
    });

    // Sidebar Toggle
    const sidebar = document.querySelector('.rk-sidebar');
    const toggleBtn = document.getElementById('rkSidebarToggle');
    const SIDEBAR_KEY = "rk_sidebar_open_v3";

    function setSidebarOpen(open){
      if(!sidebar || !toggleBtn) return;
      sidebar.style.display = open ? 'block' : 'none';
      toggleBtn.textContent = open ? '×' : '☰ Filter';
      try{ localStorage.setItem(SIDEBAR_KEY, open ? '1' : '0'); }catch(e){}
    }

    if(toggleBtn){
      toggleBtn.addEventListener('click',()=>{
        const isOpen = !(sidebar && sidebar.style.display === 'none');
        setSidebarOpen(!isOpen);
      });
      // Start immer offen
      try{ localStorage.setItem(SIDEBAR_KEY, '1'); }catch(e){}
      setSidebarOpen(true);
    }

    // Events
    document.getElementById('rkShowLines').addEventListener('change', applyFilters);
    personSel.addEventListener('change', applyFilters);
    placeSel.addEventListener('change', applyFilters);
    document.getElementById('rkYearFrom').addEventListener('input', applyFilters);
    document.getElementById('rkYearTo').addEventListener('input', applyFilters);

    document.getElementById('rkReset').addEventListener('click',()=>{
      personSel.value = "";
      placeSel.value = "";
      document.getElementById('rkYearFrom').value = "";
      document.getElementById('rkYearTo').value = "";
      document.getElementById('rkShowLines').checked = true;
      applyFilters();
    });

    // Info-Modal
    const modal = document.getElementById('rkModal');
    const openBtn = document.getElementById('rkInfo');
    const closeBtn = document.getElementById('rkModalClose');
    const chk = document.getElementById('rkDontShowAgain');
    const open = ()=>{ modal.style.display='flex'; };
    const close = ()=>{
      modal.style.display='none';
      if(chk && chk.checked){
        try{ localStorage.setItem('rk_hide_intro','1'); }catch(e){}
      }
    };
    openBtn.addEventListener('click', open);
    closeBtn.addEventListener('click', close);
    modal.addEventListener('click', (e)=>{ if(e.target===modal) close(); });
    try{ if(localStorage.getItem('rk_hide_intro')!=='1') open(); }catch(e){ open(); }

    // Sync LayerControl <-> Checkbox
    const host = window[LINES_GROUP];
    if (map && host) {
      let syncing = false;
      map.on('overlayadd', (e) => {
        if (syncing) return;
        if (e.layer === host) {
          syncing = true;
          document.getElementById('rkShowLines').checked = true;
          applyFilters();
          syncing = false;
        }
      });
      map.on('overlayremove', (e) => {
        if (syncing) return;
        if (e.layer === host) {
          syncing = true;
          document.getElementById('rkShowLines').checked = false;
          applyFilters();
          syncing = false;
        }
      });
    }

    applyFilters();
  }

  function waitForMap(tries){
    const map = getMap();
    if (map && map.whenReady && window.L){
      map.whenReady(initUi);
      return;
    }
    if (tries<=0){
      console.error("RomKorr: map not ready", MAP_NAME, map);
      return;
    }
    setTimeout(()=>waitForMap(tries-1), 60);
  }

  waitForMap(200);
})();
</script>
{% endmacro %}
""")

macro = MacroElement()
macro._template = template
macro.lines_json = json.dumps(lines_payload, ensure_ascii=False)
macro.persons_json = json.dumps(top_persons, ensure_ascii=False)
macro.places_json = json.dumps(place_payload, ensure_ascii=False)
macro.place_options_json = json.dumps(place_options, ensure_ascii=False)
macro.page_size = PAGE_SIZE_RESULTS
macro.lines_group_name = lines_group.get_name()
macro.map_name = m.get_name()

m.get_root().add_child(macro)

folium.LayerControl(collapsed=False).add_to(m)
m.save(str(OUT_HTML))

print(f"Wrote: {OUT_HTML.resolve()}")


: 